In [22]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt
import pandas as pd
import rasterio as rio
from scipy.stats import gaussian_kde

In [41]:
# Load M1 design storm subset to montecito basins. 
out = r'/Volumes/PhD/imagery/masters/output/hyper_compare/'
m1_basins = gpd.read_file(r'/Volumes/PhD/imagery/masters/QGIS/M1_design_storm_subset.shp')
m1_basins = m1_basins.to_crs(epsg=32611)
print(m1_basins.crs)
#m1_basins
print(m1_basins.columns)

EPSG:32611
Index(['BASIN_ID', 'Shape_Leng', 'Shape_Area', 'Segment_ID', 'Acc_Cl',
       'Perim_Buff', 'InsidePeri', 'OutsidePer', 'Fire_ID', 'Fire_Segme',
       'L_X1', 'L_X2', 'L_X3', 'V_X1', 'V_X2', 'Shape_Le_1', 'R', 'V_X3', 'X',
       'ExpX', 'P', 'PCl', 'PCl_Legend', 'LnV', 'Volume', 'VolMin', 'VolMax',
       'VolCl', 'VolCl_Lege', 'CombHaz', 'CombHazCl', 'CombHazCl_',
       'Mean_dNBR', 'Med_dNBR', 'PF_dNBR', 'geometry'],
      dtype='object')


In [ ]:
# Read in HdNBR & HdNBRI rasters.
# Naming convention is backwards, will be flipped in code (i.e., INPUT - idealized == S2 bands, OUTPUT - idealized == AVIRIS specific bands)


idealized_bands = r'/Volumes/PhD/imagery/masters/hyper/dnbr_aviris_warp.dat' 

S2_bands = r'/Volumes/PhD/imagery/masters/hyper/S2/AVIRIS_dNBR_idealized_t2'

#idealized_bands = rxr.open_rasterio(idealized_bands, masked=True)
idealized_bands = rxr.open_rasterio(idealized_bands, masked=True)
S2_bands = rxr.open_rasterio(S2_bands, masked=True)
print(s.rio.crs)




EPSG:32611


In [32]:
# clip to basin, and add mean, med, PF to m1 basins.
basin_list = []
mean_list = []
median_list = []
peak_frequency_list = []
i_mean_list = []
i_median_list = []
i_peak_frequency_list = []

for idx, basin in m1_basins.iterrows():
    iclipped = idealized_bands.rio.clip([basin.geometry], idealized_bands.rio.crs, drop=True)
    clipped = S2_bands.rio.clip([basin.geometry], S2_bands.rio.crs, drop=True)

    # Gather basic stats & basin id
    
    basin_list.append(basin['BASIN_ID'])
    mean_value = np.nanmean(clipped)
    median_value = np.nanmedian(clipped)

    imean_value = np.nanmean(iclipped)
    imedian_value = np.nanmedian(iclipped)

    mean_list.append(mean_value)
    median_list.append(median_value)
    i_mean_list.append(imean_value)
    i_median_list.append(imedian_value)

    # Gather kde peak frequency for both clipped and iclipped
  
    x = clipped.values.flatten()
    x = x[~np.isnan(x)]  
    
    kde = gaussian_kde(x)  # Fit KDE to non-NaN values

    # Define a grid of values for evaluation
    x_grid = np.linspace(x.min(), x.max(), 1000)

    # Evaluate KDE on the grid
    kde_values = kde(x_grid)

    # Find the peak value
    peak_index = kde_values.argmax() 
    peak_frequency = x_grid[peak_index]

    y = iclipped.values.flatten()
    y = y[~np.isnan(y)]  # Remove NaN values
    
    ikde = gaussian_kde(y)  # Fit KDE to non-NaN values

    # Define a grid of values for evaluation
    y_grid = np.linspace(y.min(), y.max(), 1000)

    # Evaluate KDE on the grid
    ikde_values = kde(y_grid)

    # Find the peak value
    ipeak_index = ikde_values.argmax() 
    ipeak_frequency = y_grid[ipeak_index]

    peak_frequency_list.append(peak_frequency)
    i_peak_frequency_list.append(ipeak_frequency)
    


In [48]:
m1_basins["HMean_dNBR"] = mean_list
m1_basins["HMed_dNBR"] = median_list
m1_basins["HPF_dNBR"] = peak_frequency_list
m1_basins["iHMean_dNBR"] = i_mean_list
m1_basins["iHMed_dNBR"] = i_median_list
m1_basins["iHPF_dNBR"] = i_peak_frequency_list
# Save to shapefile
m1_basins.to_file(out + 'M1_basins_dNBR_stats.shp')


/var/folders/mm/sjj0b1k91z39pkxfzn0rj_1h0000gn/T/ipykernel_73474/3212865372.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  m1_basins.to_file(out + 'M1_basins_dNBR_stats.shp')
/Users/samueltaylor/Documents/GitHub/PHA_burn_severity/.venv/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'iHMean_dNBR' to 'iHMean_dNB'
  ogr_write(


In [49]:
test = gpd.read_file(r'/Volumes/PhD/imagery/masters/output/hyper_compare/M1_basins_dNBR_stats.shp')
print(test.columns)


Index(['BASIN_ID', 'Shape_Leng', 'Shape_Area', 'Segment_ID', 'Acc_Cl',
       'Perim_Buff', 'InsidePeri', 'OutsidePer', 'Fire_ID', 'Fire_Segme',
       'L_X1', 'L_X2', 'L_X3', 'V_X1', 'V_X2', 'Shape_Le_1', 'R', 'V_X3', 'X',
       'ExpX', 'P', 'PCl', 'PCl_Legend', 'LnV', 'Volume', 'VolMin', 'VolMax',
       'VolCl', 'VolCl_Lege', 'CombHaz', 'CombHazCl', 'CombHazCl_',
       'Mean_dNBR', 'Med_dNBR', 'PF_dNBR', 'HMean_dNBR', 'HMed_dNBR',
       'HPF_dNBR', 'iHMean_dNB', 'iHMed_dNBR', 'iHPF_dNBR', 'geometry'],
      dtype='object')
